# Downloaded Dataset Directly from

Github Link:
https://github.com/yastrebksv/TennisCourtDetector?tab=readme-ov-file

Google Drive Link:
https://drive.google.com/file/d/1lhAaeQCmk2y440PmagA0KmIVBIysVMwu/view

In [ ]:
!unzip tennis_court_det_dataset.zip

# Start Code

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np
import random

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [6]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)), #Resize images to 224x224
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.0485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2] #original image size

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps


            


In [ ]:
train_dataset = KeypointsDataset("data/images", "data/data_train.json")
val_dataset = KeypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Create model

In [ ]:
model = models.resnet50(pretrained=True)
#fine tuning resnet50 model (was pretrained)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer of the neural network

In [ ]:
model = model.to(device)

# Train Model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
num_epochs=20
for epoch in range(num_epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, iter {i+1}/{len(train_loader)}, Loss: {loss.iter()}")





In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")